In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd

In [3]:
data = pd.read_csv('preprocessed_data.csv')
data.dropna(subset=['cleaned_text'], inplace=True)
data['cleaned_text'] = data['cleaned_text'].astype(str) #convert 'cleaned_text' to string

In [ ]:
# prepare feature matrix X and the target y
X = data['cleaned_text']
y = data['label'].astype(int)

# tf-idf
vectorizer = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True)
X_vectorized = vectorizer.fit_transform(X)


# split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


# Convert to dense array for Tensorflow
X_train = X_train.toarray()
X_test = X_test.toarray()

In [ ]:
# Discriminator Model
def build_discriminator(input_shape):
    text_input = Input(shape=(input_shape,))
    d = Dense(512)(text_input)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dense(256)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dense(128)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Dropout(0.4)(d)
    d = Dense(64)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # Two outputs, one for the real/fake classification, one for the actual class
    real_or_fake = Dense(1, activation='sigmoid')(d)
    class_label = Dense(1, activation='sigmoid')(d) # Binary classification
    return Model(text_input, [real_or_fake, class_label])

In [ ]:
# Generator Model
def build_generator(latent_dim, output_shape):
    noise = Input(shape=(latent_dim,))
    g = Dense(256)(noise)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dense(512)(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dense(1024)(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Dense(output_shape, activation='linear')(g) # No activation, linear to mimic TF-IDF
    return Model(noise, g)


In [ ]:
X_train.shape[1]